# Make parameter submission scripts for HPC 

## Carrying capacity sweep

Author: Juvid Aryaman

In [1]:
import numpy as np
import pandas as pd
import mitonetworks.det as mtd
import mitonetworks.stoch as mts

In [2]:
nominal_params = {
    'gamma':0.03785142857142858,
	'beta':33.12,
	'kappa':11.662903457629223,
	'b':1.2416523075924095e-05,
	'mu':0.023,
	'xi':0.0,
	'delta':1.0
}

In [3]:
c_file_param_ordering_convention = ['xi','beta','gamma','kappa','b','mu','delta']

In [4]:
hpc_workdir = '$WORK/networks/sing_birth_fus/kappa_sw'

In [5]:
c_filename = 'closed_loop_deg_single'

In [6]:
hpc_jobname = 'kappa_sw'

In [7]:
ks = mts.SubmitHPCSoluableFeedbackControl(ss_definition=mtd.E_linear_feedback_control_ss,
                                 nominal_params=nominal_params,hpc_workdir=hpc_workdir,
                                c_filename=c_filename, c_file_param_ordering_convention=c_file_param_ordering_convention,
                                hpc_jobname=hpc_jobname)

In [8]:
ks.param_convention

['ws', 'wf', 'ms', 'mf']

In [9]:
kappa_rel_sp = [0,50,100,200,300]
params = nominal_params.copy()

master_script = open(ks.out_dir+"/master.sh","w")
# Add the .c compilation line to the master script
master_script.write('gcc -o3 {0}.c -lm -o {0}.ce\n'.format(ks.c_filename))

# Output dataframe with parametriations
param_df =  []

for i, kappa_r in enumerate(kappa_rel_sp):
    params['kappa'] = kappa_r*nominal_params['kappa']
    ics, h_opt, err = ks.find_ss_h_target(params)
    
    param_dict = params.copy()
    for k, param in enumerate(ks.param_convention): param_dict[param+'_init'] = ics[k]
    param_df.append(pd.DataFrame(param_dict, index=[0]))

    if err == 1:
        ks.make_submission_script(i,ics,params)
        # Add the submission to a master script
        master_script.write('echo "{}"\n'.format(i))
        master_script.write('mkdir -p {0}/p{1}\n'.format(ks.hpc_workdir,i))
        master_script.write("qsub submission_{}.pbs\n".format(i))
        master_script.write('sleep 0.5\n')
    else:
        raise Exception('SS not found!')
master_script.close()
param_df = pd.concat(param_df, ignore_index=True)
param_df.to_csv(ks.out_dir+'/param_sweep_vals.csv',index=False)    

In [10]:
param_df

,b,beta,delta,gamma,kappa,mf_init,ms_init,mu,wf_init,ws_init,xi
0,0.000012,33.12,1.0,0.037851,0.000000,154,138,0.023,362,324,0.0
1,0.000012,33.12,1.0,0.037851,583.145173,374,178,0.023,872,415,0.0
2,0.000012,33.12,1.0,0.037851,1166.290346,567,195,0.023,1327,456,0.0
3,0.000012,33.12,1.0,0.037851,2332.580692,940,214,0.023,2189,499,0.0
4,0.000012,33.12,1.0,0.037851,3498.871037,1300,224,0.023,3032,522,0.0
